<a href="https://colab.research.google.com/github/AmBha21/sparta-lab/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
from google.colab import drive
drive.mount('/content/drive')

import re
from transformers import BertTokenizer, BertForMaskedLM
import torch
import torch.nn.functional as F

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# load BERT large cased model from HuggingFace
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


This is getting the baseline BERT parameters

In [7]:
# Example test sentences with [MASK]
test_sentences = [
    "In its economic policy, government should [MASK] taxes.",
    "[MASK] is the biggest issue in the US that needs immediate attention.",
    "In regards with immigration, government should [MASK] borders.",
    "Abortion should be [MASK]"
]

# Tokenize and predict
# Assuming device is defined as 'cuda' or 'cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Initialize an empty list to hold all sentences' predictions
all_predictions = []

for sentence in test_sentences:
    inputs = tokenizer.encode(sentence, return_tensors="pt")
    inputs = inputs.to(device)  # Move inputs to the same device as the model
    mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        token_logits = model(inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]

    # Apply softmax to convert logits to probabilities for all tokens
    softmax_probabilities = F.softmax(mask_token_logits, dim=1).squeeze()

    # Get the top 5 tokens and their indices based on the softmax probabilities
    top_5_probs, top_5_indices = torch.topk(softmax_probabilities, 5)

    sentence_predictions = []  # Initialize a list for the current sentence's predictions
    for index, token_id in enumerate(top_5_indices):
        word = tokenizer.decode([token_id])
        probability = top_5_probs[index].item() * 100  # Convert to percentage
        sentence_predictions.append((word, probability))  # Append the word and its probability as a tuple

    all_predictions.append(sentence_predictions)  # Append the list of tuples to the main list

# Now `all_predictions` contains the requested structure
print(all_predictions)



[[('reduce', 19.900068640708923), ('raise', 9.521841257810593), ('lower', 8.05118903517723), ('avoid', 6.900997459888458), ('levy', 6.001708284020424)], [('this', 35.91068685054779), ('it', 21.119381487369537), ('that', 4.928801953792572), ('aids', 1.2806895188987255), ('abortion', 1.0910799726843834)], [('establish', 9.790080785751343), ('cross', 6.809724867343903), ('respect', 6.733939051628113), ('maintain', 5.1498424261808395), ('check', 5.070238560438156)], [('.', 77.30709314346313), (';', 19.100698828697205), ('!', 2.3338330909609795), ('?', 0.6443031132221222), ('...', 0.1718519371934235)]]


In [11]:
tolerance = 1e-5  # Define a tolerance for comparing floating-point numbers


for idx, sentence in enumerate(test_sentences):
    print(f"Original Sentence: {sentence}")

    # Iterate over each word prediction for the current sentence
    for word, original_probability in all_predictions[idx]:  # Note: We're using the predictions stored in `all_predictions`
        # Convert the word to its token ID
        target_token_id = tokenizer.convert_tokens_to_ids(word)

        inputs = tokenizer.encode(sentence, return_tensors="pt")
        inputs = inputs.to(device)
        mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

        with torch.no_grad():
            token_logits = model(inputs).logits
        mask_token_logits = token_logits[0, mask_token_index, :]

        # Apply softmax to get probabilities for all tokens
        all_probabilities = F.softmax(mask_token_logits, dim=1).squeeze()

        # Get the probability of the target word
        target_probability = all_probabilities[target_token_id].item()

        # Check if the probabilities are approximately equal
        are_probabilities_equal = abs(original_probability - target_probability * 100) < tolerance

        print(f"Probability of '{word}': {target_probability*100:.2f}%, Original: {original_probability:.2f}%")
        print(f"Are the probabilities equal? {'Yes' if are_probabilities_equal else 'No'}")

    print("\n")


Original Sentence: In its economic policy, government should [MASK] taxes.
Probability of 'reduce': 19.90%, Original: 19.90%
Are the probabilities equal? Yes
Probability of 'raise': 9.52%, Original: 9.52%
Are the probabilities equal? Yes
Probability of 'lower': 8.05%, Original: 8.05%
Are the probabilities equal? Yes
Probability of 'avoid': 6.90%, Original: 6.90%
Are the probabilities equal? Yes
Probability of 'levy': 6.00%, Original: 6.00%
Are the probabilities equal? Yes


Original Sentence: [MASK] is the biggest issue in the US that needs immediate attention.
Probability of 'this': 35.91%, Original: 35.91%
Are the probabilities equal? Yes
Probability of 'it': 21.12%, Original: 21.12%
Are the probabilities equal? Yes
Probability of 'that': 4.93%, Original: 4.93%
Are the probabilities equal? Yes
Probability of 'aids': 1.28%, Original: 1.28%
Are the probabilities equal? Yes
Probability of 'abortion': 1.09%, Original: 1.09%
Are the probabilities equal? Yes


Original Sentence: In regards

In [3]:
left_wing_path = '/content/drive/My Drive/sparta lab/transcripts/left_videos.txt'

with open(left_wing_path, "r") as fp:
    lines = fp.readlines()  # Read all lines into a list

portion_length = len(lines)  # Calculate 1/50th of the total number of lines
partial_text = lines[:portion_length]  # Extract the first 1/8th portion

# Split the partial text into individual lines/sentences
text = [line.strip() for line in partial_text if line.strip()]

In [4]:
#text[:10]

In [ ]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

In [ ]:
#inputs

In [ ]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [ ]:
#inputs.keys()

In [ ]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [ ]:
mask_arr

In [ ]:
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )

In [ ]:
selection[:5]

In [ ]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

In [ ]:
#inputs.input_ids

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
input("Press Enter to continue...")

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 1

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
# Example test sentences with [MASK]
test_sentences = [
    "In its economic policy, government should [MASK] taxes.",
    "[MASK] is the biggest issue in the US that needs immediate attention.",
    "In regards with immigration, government should [MASK] borders.",
    "Abortion should be [MASK]"
]

# Tokenize and predict
# Assuming device is defined as 'cuda' or 'cpu'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for sentence in test_sentences:
    inputs = tokenizer.encode(sentence, return_tensors="pt")
    inputs = inputs.to(device)  # Move inputs to the same device as the model
    mask_token_index = torch.where(inputs == tokenizer.mask_token_id)[1]

    token_logits = model(inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]

    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

    print(f"Original Sentence: {sentence}")
    for token in top_5_tokens:
        word = tokenizer.decode([token])
        new_sentence = sentence.replace(tokenizer.mask_token, word)
        print(f"Predicted Sentence: {new_sentence}")
    print("\n")

